In [1]:
import sys
import os
from pathlib import Path
from glob import glob

In [2]:
sys.path.append('..')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

In [4]:
from experiments.eval_features import eval_features

In [5]:
PATH_ROOT = Path(os.getcwd()).absolute().parent
print(PATH_ROOT)

/home/lukec/workspace/baard_v4


In [6]:
def get_result_one_detector(detector_name,
                            data_name,
                            model_name,
                            attack_name,
                            eps,
                            path_feature,
                            path_output):  
    """Get AUC and TPR as a DataFrame for one feature."""
    path_input = os.path.join(path_feature, f'{data_name}-{model_name}', detector_name, attack_name)
    file_adv = f'{detector_name}-{data_name}-{attack_name}-{eps}.pt'
    file_clean = f'{detector_name}-{data_name}-{attack_name}-clean.pt'
    _, df_auc_tpr = eval_features(path_input, path_output, file_clean, file_adv)
    df_auc_tpr = (df_auc_tpr * 100).round(1)
    df_auc_tpr['data'] = data_name
    df_auc_tpr['detector'] = detector_name
    df_auc_tpr['attack'] = attack_name
    df_auc_tpr['eps'] = eps
    df_auc_tpr = df_auc_tpr[['data', 'detector', 'attack', 'eps', 'auc', '1fpr', '5fpr', '10fpr']]
    return df_auc_tpr


In [7]:
data_list = ['banknote', 'BC']
model_list = ['SVM', 'DecisionTree']
attack_list = ['PGD-Linf', 'DecisionTreeAttack']
eps_list = {
    'PGD-Linf': [0.2, 0.6],
    'DecisionTreeAttack': [0],
}
detectors = [
    'SklearnBAARD',
    'SklearnRegionBasedClassifier',
]
detector_mapping = {
    'SklearnBAARD': 'BAARD',
    'SklearnRegionBasedClassifier': 'RC',
}
eps_mapping = {
    0.2: 'Low',
    0.6: 'High',
    0: 'NA',
}

In [8]:
seed_list = [
    188283,
    292478,
    382347,
    466364,
    543597,
]

df_list = []
for s in seed_list:
    path_results = os.path.join(PATH_ROOT, 'results', f'exp{s}')
    path_output = os.path.join(path_results, 'roc')
    print('INPUTS:', path_results)

    if not os.path.exists(path_output):
        os.makedirs(path_output)

    _df = pd.DataFrame()
    for _data_name in data_list:
        for _model_name, _att_name in zip(model_list, attack_list):
            for _eps in eps_list[_att_name]:
                for _detector_name in detectors:
                    _df_row = get_result_one_detector(
                        _detector_name,
                        _data_name,
                        _model_name,
                        _att_name,
                        _eps,
                        path_results,
                        path_output,
                    )
                    _df = pd.concat([_df, _df_row], ignore_index=True)

    _df['detector'] = _df['detector'].map(detector_mapping)
    _df['eps'] = _df['eps'].map(eps_mapping)
    _df = _df.sort_values(
        by=['data', 'attack', 'eps', 'detector'],
        ascending=[False, False, False, False]
    )
    _df.to_csv(os.path.join(path_output, 'graybox_sklearn.csv'), index=False)
    df_list.append(_df)


INPUTS: /home/lukec/workspace/baard_v4/results/exp188283
INPUTS: /home/lukec/workspace/baard_v4/results/exp292478
INPUTS: /home/lukec/workspace/baard_v4/results/exp382347


/home/lukec/workspace/baard_v4/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


INPUTS: /home/lukec/workspace/baard_v4/results/exp466364


/home/lukec/workspace/baard_v4/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


INPUTS: /home/lukec/workspace/baard_v4/results/exp543597


/home/lukec/workspace/baard_v4/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lukec/workspace/baard_v4/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

In [9]:
display_columns = ['data', 'detector', 'attack', 'eps', 'auc', '5fpr']

for i, _df in enumerate(df_list):
    print(f'Exp: {i+1}')
    print(_df[display_columns])

Exp: 1
        data detector              attack   eps    auc   5fpr
1   banknote       RC            PGD-Linf   Low   78.4   30.2
0   banknote    BAARD            PGD-Linf   Low   98.2   92.0
3   banknote       RC            PGD-Linf  High   98.5  100.0
2   banknote    BAARD            PGD-Linf  High  100.0  100.0
5   banknote       RC  DecisionTreeAttack    NA   92.6   58.7
4   banknote    BAARD  DecisionTreeAttack    NA   94.8   82.9
7         BC       RC            PGD-Linf   Low   63.0    0.0
6         BC    BAARD            PGD-Linf   Low   79.0   18.9
9         BC       RC            PGD-Linf  High   74.2    0.0
8         BC    BAARD            PGD-Linf  High   60.6    2.7
11        BC       RC  DecisionTreeAttack    NA   97.6   89.5
10        BC    BAARD  DecisionTreeAttack    NA   99.2   97.1
Exp: 2
        data detector              attack   eps    auc   5fpr
1   banknote       RC            PGD-Linf   Low   81.9   57.8
0   banknote    BAARD            PGD-Linf   Low   94.9  

In [ ]:
df_mean = pd.concat(df_list, ignore_index=True).groupby(['data', 'detector', 'attack', 'eps']).mean()
df_std = pd.concat(df_list, ignore_index=True).groupby(['data', 'detector', 'attack', 'eps']).std()

df_mean.round(2).to_csv(os.path.join(PATH_ROOT, 'plots', 'graybox_sklearn_mean.csv'))
df_std.round(2).to_csv(os.path.join(PATH_ROOT, 'plots', 'graybox_sklearn_std.csv'))